In [ ]:
# ==========================================================================================
#                      PROYECTO FINAL - CLASIFICACION SPAM/NOT SPAM
#                              Ismael Hernandez Clemente
# ==========================================================================================
#
# CONTEXTO:
# Este es mi modelo final (V6) despues de probar 5 versiones diferentes.
# He aprendido que lo simple funciona mejor que lo complejo.
# 
# ITERACIONES PREVIAS:
# - V1: LSTM basico, funcionaba pero mucho overfitting
# - V2: Mejor score publico (0.8885) pero segui teniendo overfitting
# - V3: Regularizacion fuerte, bajo el overfitting pero bajo un poco el MCC
# - V4: Probe DistilBERT... fue un desastre total (0.64 de MCC)
# - V5: CNN+LSTM hibrido, demasiado complejo y no mejoro nada
# - V6: Vuelvo a V3 con pequeños ajustes, el mas estable
#
# POR QUE ELEGI V6 Y NO V2:
# Si, V2 tiene mejor score publico (0.8885 vs 0.87)
# Pero V6 tiene mucho menos overfitting (0.09 vs 0.166)
# Prefiero un modelo que generalice bien a uno que solo brille en el leaderboard publico
# En el ranking privado creo que V6 sera mas estable
#
# ARQUITECTURA:
# Simplemente un LSTM bidireccional con mucha regularizacion
# Nada de fancy stuff, solo lo que funciona
#
# ==========================================================================================

In [ ]:
# Configuracion del entorno para que TensorFlow no llene la consola de warnings
# Estos warnings no aportan nada util, solo molestan
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'  # Evita warnings de protobuf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Solo errores, no warnings
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'  # Desactiva optimizaciones que dan warnings

In [ ]:
# Imports basicos para trabajar con datos
import pandas as pd  # Para manejar los CSV
import numpy as np  # Operaciones numericas
import matplotlib.pyplot as plt  # Graficos
import seaborn as sns  # Graficos bonitos
import warnings
warnings.filterwarnings('ignore')  # Quito todos los warnings molestos

# Semilla para reproducibilidad - IMPORTANTE
# Si no pongo esto, cada vez que ejecute tendre resultados diferentes
seed = 42
np.random.seed(seed)

# Imports de TensorFlow/Keras para el modelo
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer  # Para convertir texto a numeros
from tensorflow.keras.preprocessing.sequence import pad_sequences  # Para que todas las secuencias tengan el mismo tamaño
from tensorflow.keras.models import Model  # Para construir el modelo
from tensorflow.keras.layers import (
    Input,  # Capa de entrada
    Embedding,  # Convierte palabras en vectores
    LSTM,  # Red recurrente para secuencias
    Bidirectional,  # Lee la secuencia en ambas direcciones
    Dense,  # Capa densa normal
    Dropout,  # Apaga neuronas aleatoriamente (regularizacion)
    GlobalMaxPooling1D,  # No lo uso al final
    SpatialDropout1D,  # Dropout especial para embeddings
    Attention,  # No lo uso, causaba overfitting
    Permute,  # No lo uso
    Multiply  # No lo uso
)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau  # Callbacks para el entrenamiento
from tensorflow.keras.regularizers import l2  # Regularizacion L2

# Mas semillas para TensorFlow (para que sea 100% reproducible)
tf.random.set_seed(seed)
keras.utils.set_random_seed(seed)

# Imports de sklearn para metricas
from sklearn.metrics import matthews_corrcoef, classification_report, confusion_matrix  # Metricas de evaluacion
from sklearn.model_selection import train_test_split  # Para dividir train/val

# Configuracion de pandas para ver mejor los datos
pd.set_option('display.max_rows', 36)
pd.set_option("display.max_colwidth", 150)

In [ ]:
# HIPERPARAMETROS - Estos son los valores que funcionaron mejor despues de 6 iteraciones

# Preprocesamiento de texto
MAX_WORDS = 10000  # Uso solo las 10k palabras mas frecuentes (el dataset tiene 46k)
                   # Probe con 20k y no mejoro, las palabras raras solo añaden ruido
MAX_LEN = 200  # Corto/relleno los textos a 200 tokens
               # Probe con 250 y causaba overfitting
EMBEDDING_DIM = 100  # Dimension de los vectores de palabras
                     # 100 es suficiente, 128 causaba overfitting

# Arquitectura del modelo
LSTM_UNITS = 64  # Unidades LSTM (64*2=128 por ser bidireccional)
                 # V1 tenia 128 (overfitting), V2 tenia 96, 64 es el optimo
DENSE_UNITS = 32  # Neuronas en la capa densa
                  # Lo justo para combinar las features sin sobreajustar

# Regularizacion - CLAVE para controlar overfitting
SPATIAL_DROPOUT = 0.4  # Dropout agresivo en embeddings
                       # Apaga mapas completos de features
DROPOUT_RATE = 0.7  # Dropout MUY agresivo en capa densa
                    # Apaga el 70% de las conexiones - suena brutal pero es necesario
L2_REG = 6e-4  # Regularizacion L2 en todas las capas con pesos
               # V3 usaba 5e-4, subi a 6e-4 para mejor control
               # Penaliza pesos grandes, fuerza al modelo a distribuir importancia

# Entrenamiento
BATCH_SIZE = 32  # Tamaño de lote estandar
EPOCHS = 50  # Maximo de epochs (pero EarlyStopping para antes)
VALIDATION_SPLIT = 0.2  # 80% train, 20% validacion
LEARNING_RATE = 5e-4  # Learning rate conservador para convergencia estable
                      # V1 usaba 1e-3 (demasiado agresivo)
CLIPNORM = 1.0  # Gradient clipping para evitar que exploten los gradientes
                # Importante en LSTMs



In [ ]:
# Cargo los datos de entrenamiento
# index_col="row_id" para usar esa columna como indice
train = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/train.csv", index_col="row_id")

In [ ]:
# PREPROCESAMIENTO DEL TEXTO
# Deliberadamente simple - no quito stopwords ni hago stemming porque pueden ser importantes para detectar spam

# Creo el tokenizer que convertira texto a numeros
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token='<OOV>')  # oov_token para palabras desconocidas
tokenizer.fit_on_texts(train['text'])  # Aprende el vocabulario del texto de entrenamiento

# Convierto los textos a secuencias de numeros
X_train_seq = tokenizer.texts_to_sequences(train['text'])

# Padding: ajusto todas las secuencias a la misma longitud
# padding='post' -> relleno al final (preservo el inicio del mensaje)
# truncating='post' -> corto por el final si es muy largo (preservo el inicio)
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post', truncating='post')

# Etiquetas (0 = NOT SPAM, 1 = SPAM)
y_train = train['spam_label'].values

# Divido en train/validacion (80/20)
# stratify=y_train para mantener la proporcion de clases (75% NOT SPAM, 25% SPAM)
X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train_pad, y_train, 
    test_size=VALIDATION_SPLIT, 
    random_state=seed, 
    stratify=y_train  # Importante: mantiene el balance de clases
)

# Muestro info basica
print(f"Muestras entrenamiento: {len(X_train_final)}")
print(f"Muestras validacion: {len(X_val)}")
print(f"Proporcion SPAM: {y_train.mean():.2%}")

In [ ]:
def build_v6_model():
    """
    Construyo mi modelo V6 - arquitectura simple pero efectiva
    
    Flujo:
    Input -> Embedding -> Spatial Dropout -> Bi-LSTM -> Dense -> Dropout -> Output
    
    La clave esta en la regularizacion agresiva (dropout 0.7 + L2 6e-4)
    """
    
    # Capa de entrada: secuencias de 200 tokens
    inputs = Input(shape=(MAX_LEN,), name='input_sequences')
    
    # Embedding: convierte tokens a vectores de 100 dimensiones
    # Entreno los embeddings desde cero (no uso preentrenados)
    x = Embedding(
        input_dim=MAX_WORDS,  # Vocabulario de 10k palabras
        output_dim=EMBEDDING_DIM,  # Vectores de 100 dimensiones
        input_length=MAX_LEN,
        name='embedding'
    )(inputs)
    
    # Spatial Dropout: apaga mapas completos de features (no elementos individuales)
    # Esto previene que el modelo dependa de embeddings especificos
    x = SpatialDropout1D(SPATIAL_DROPOUT, name='spatial_dropout')(x)
    
    # LSTM Bidireccional: lee el texto en ambas direcciones
    # 64 units * 2 directions = 128 dimensiones efectivas
    # L2 en kernel, recurrent y bias: regularizacion total
    lstm_out = Bidirectional(
        LSTM(
            LSTM_UNITS,
            kernel_regularizer=l2(L2_REG),  # Regulariza pesos input->hidden
            recurrent_regularizer=l2(L2_REG),  # Regulariza pesos hidden->hidden
            bias_regularizer=l2(L2_REG),  # Regulariza bias
            return_sequences=False  # Solo quiero el output final, no toda la secuencia
        ),
        name='bidirectional_lstm'
    )(x)
    
    # Capa densa: combina las features del LSTM
    # ReLU como activacion (estandar)
    dense = Dense(
        DENSE_UNITS,
        activation='relu',
        kernel_regularizer=l2(L2_REG),  # Mas regularizacion L2
        bias_regularizer=l2(L2_REG),
        name='dense_classifier'
    )(lstm_out)
    
    # Dropout brutal: 0.7 = apaga el 70% de las conexiones
    # Suena excesivo pero es necesario para evitar overfitting
    dense = Dropout(DROPOUT_RATE, name='dropout')(dense)
    
    # Output: sigmoid para probabilidad entre 0 y 1
    # >0.5 = SPAM, <0.5 = NOT SPAM
    outputs = Dense(1, activation='sigmoid', name='output')(dense)
    
    # Construyo el modelo
    model = Model(inputs=inputs, outputs=outputs, name='V6_LSTM_Final')
    return model

# Creo el modelo
model = build_v6_model()

# Optimizador AdamW con gradient clipping
# AdamW es Adam mejorado con weight decay
optimizer = keras.optimizers.AdamW(
    learning_rate=LEARNING_RATE,  # 5e-4, conservador pero estable
    weight_decay=1e-4,  # Weight decay adicional
    clipnorm=CLIPNORM  # Limita la norma del gradiente a 1.0 (evita explosiones)
)

# Compilo el modelo
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',  # Loss estandar para clasificacion binaria
    metrics=[
        'accuracy',  # Porcentaje de aciertos
        keras.metrics.Precision(name='precision'),  # De los que predigo SPAM, cuantos son SPAM
        keras.metrics.Recall(name='recall'),  # De los SPAM reales, cuantos detecto
        keras.metrics.AUC(name='auc')  # Area bajo la curva ROC
    ]
)

# Muestro resumen del modelo
print("\n" + "="*80)
print("MODELO V6 CONSTRUIDO")
print("="*80)
model.summary()
print("="*80)

In [ ]:

callbacks = [
    # EarlyStopping: para si val_loss no mejora
    EarlyStopping(
        monitor='val_loss',  # Monitoreo val_loss
        patience=2,  # Si no mejora en 2 epochs, paro
        restore_best_weights=True,  # Al final, uso los pesos de la mejor epoch
        verbose=1
    ),
    
    # ModelCheckpoint: guarda el mejor modelo
    ModelCheckpoint(
        'best_spam_model_v6.keras',  # Nombre del archivo
        monitor='val_loss',  # Guardo cuando val_loss mejora
        save_best_only=True,  # Solo guardo si mejora
        verbose=1
    ),
    
    # ReduceLROnPlateau: reduce learning rate si val_loss se estanca
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,  # Divide LR por 2
        patience=1,  # Espera solo 1 epoch antes de reducir
        min_lr=1e-6,  # LR minimo
        verbose=1
    )
]

# ENTRENO EL MODELO
print("\nIniciando entrenamiento...")
history = model.fit(
    X_train_final, y_train_final,  # Datos de entrenamiento
    batch_size=BATCH_SIZE,  # Lotes de 32
    epochs=EPOCHS,  # Maximo 50 epochs (pero EarlyStopping parara antes)
    validation_data=(X_val, y_val),  # Datos de validacion
    callbacks=callbacks,  # Uso los callbacks definidos arriba
    verbose=1  # Muestro progreso
)
print("Entrenamiento completado!")

In [ ]:
# EVALUACION DEL MODELO EN VALIDACION

# Obtengo probabilidades de prediccion
y_pred_proba = model.predict(X_val, batch_size=BATCH_SIZE, verbose=0).flatten()

# Convierto probabilidades a clases (threshold 0.5)
best_threshold = 0.5  # Estandar: >0.5 = SPAM, <0.5 = NOT SPAM
y_pred = (y_pred_proba > best_threshold).astype(int)

# Calculo MCC (metrica principal de la competicion)
mcc_val = matthews_corrcoef(y_val, y_pred)

print("="*80)
print("EVALUACION - METRICAS DE VALIDACION")
print("="*80)
print(f"MCC en validacion (threshold 0.5): {mcc_val:.4f}")

# Analizo el entrenamiento para detectar overfitting
final_epoch = len(history.history['loss'])  # Cuantas epochs se entrenaron
train_loss_final = history.history['loss'][-1]  # Loss final de train
val_loss_final = history.history['val_loss'][-1]  # Loss final de validacion
train_acc_final = history.history['accuracy'][-1]  # Accuracy final de train
val_acc_final = history.history['val_accuracy'][-1]  # Accuracy final de validacion

# Delta de overfitting: diferencia entre val_loss y train_loss
# Si es muy grande, hay overfitting
overfitting_delta = val_loss_final - train_loss_final

print(f"\nEntrenamiento detenido en epoch: {final_epoch}")
print(f"Train Loss: {train_loss_final:.4f} | Val Loss: {val_loss_final:.4f}")
print(f"Train Acc:  {train_acc_final:.4f} | Val Acc:  {val_acc_final:.4f}")
print(f"Delta Overfitting: {overfitting_delta:.4f}")

# Interpreto el overfitting
if overfitting_delta > 0.15:
    print("\nOVERFITTING DETECTADO (delta > 0.15)")
    print("-> Necesito MAS regularizacion")
elif overfitting_delta < 0.05:
    print("\nSin overfitting significativo (delta < 0.05)")
    print("-> Podria permitirme MENOS regularizacion")
else:
    print("\nOverfitting CONTROLADO (delta entre 0.05-0.15)")
    print("-> Balance perfecto!")

# Analizo la distribucion de probabilidades
print(f"\nDistribucion de probabilidades:")
print(f"  Media: {y_pred_proba.mean():.4f}")
print(f"  Desv std: {y_pred_proba.std():.4f}")
print(f"  Min: {y_pred_proba.min():.4f}")
print(f"  Max: {y_pred_proba.max():.4f}")

# Analizo por clase
spam_probs = y_pred_proba[y_val == 1]  # Probabilidades de mensajes SPAM reales
notspam_probs = y_pred_proba[y_val == 0]  # Probabilidades de mensajes NOT SPAM reales

print(f"\nProbabilidad media clase SPAM: {spam_probs.mean():.4f} (deberia ser > 0.5)")
print(f"Probabilidad media clase NOT SPAM: {notspam_probs.mean():.4f} (deberia ser < 0.5)")

# Separacion de clases: cuanto separa el modelo las dos clases
separation = spam_probs.mean() - notspam_probs.mean()
print(f"Separacion de clases: {separation:.4f} (cuanto mayor, mejor)")

if separation < 0.3:
    print("-> BAJA separacion, el modelo no distingue bien")
elif separation > 0.6:
    print("-> BUENA separacion, el modelo distingue claramente")

# Reporte de clasificacion completo
print("\n" + "="*80)
print("CLASSIFICATION REPORT:")
print("="*80)
print(classification_report(y_val, y_pred, target_names=['Not SPAM', 'SPAM']))
print("="*80)

In [ ]:
# PREDICCIONES EN TEST PARA SUBMISSION

# Cargo datos de test
test = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/test.csv", index_col="row_id")

# Preproceso igual que train
X_test_seq = tokenizer.texts_to_sequences(test['text'])  # Texto a numeros
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post', truncating='post')  # Padding

# Predigo
y_test_proba = model.predict(X_test_pad, batch_size=BATCH_SIZE, verbose=0).flatten()  # Probabilidades
y_test_pred = (y_test_proba > best_threshold).astype(int)  # Convierto a clases

# Creo el submission
submission = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/sample_submission.csv")
submission["spam_label"] = y_test_pred  # Reemplazo con mis predicciones
submission.to_csv('submission.csv', index=False)  # Guardo

print(f"\nSubmission creado: {len(y_test_pred)} predicciones")
print(f"Threshold usado: {best_threshold:.2f}")
print(f"Predicciones SPAM: {y_test_pred.sum()} ({y_test_pred.mean():.2%})")
print(f"Predicciones NOT SPAM: {len(y_test_pred) - y_test_pred.sum()} ({(1-y_test_pred.mean()):.2%})")

In [ ]:
# RESUMEN Y DIAGNOSTICO AUTOMATICO
# Este codigo analiza las metricas y me dice que hacer en la proxima iteracion

print("\n" + "="*80)
print("RESUMEN DE RESULTADOS")
print("="*80)
print(f"Val MCC: {mcc_val:.4f} | Overfitting Delta: {overfitting_delta:.4f} | Separacion: {separation:.4f}")
print("")

# Arbol de decision basado en las metricas
if overfitting_delta > 0.15:
    print("DIAGNOSTICO: OVERFITTING SEVERO")
    print("ACCION:")
    print("   1. Aumentar L2: 6e-4 -> 8e-4")
    print("   2. Aumentar Dropout: 0.7 -> 0.75")
    print("   3. Reducir LSTM units: 64 -> 56")
    
elif overfitting_delta < 0.05 and mcc_val < 0.88:
    print("DIAGNOSTICO: UNDERFIT - Demasiada regularizacion")
    print("ACCION:")
    print("   1. Reducir L2: 6e-4 -> 4e-4")
    print("   2. Reducir Dropout: 0.7 -> 0.6")
    print("   3. Aumentar LSTM units: 64 -> 80")
    
elif separation < 0.3:
    print("DIAGNOSTICO: BAJA CAPACIDAD DISCRIMINATIVA")
    print("ACCION:")
    print("   1. Aumentar MAX_LEN: 200 -> 220")
    print("   2. Aumentar EMBEDDING_DIM: 100 -> 128")
    print("   3. Mantener regularizacion actual")
    
elif mcc_val >= 0.87 and overfitting_delta <= 0.12:
    print("DIAGNOSTICO: MODELO BALANCEADO")
    print("ACCION:")
    print("   1. Este es probablemente el mejor resultado alcanzable")
    print("   2. Usar ESTE modelo para submission")
    print("   3. Si el score de test es peor, el problema es el dataset, no el modelo")

else:
    print("DIAGNOSTICO: AJUSTE FINO NECESARIO")
    print("ACCION:")
    print("   1. Probar L2 entre 5e-4 y 7e-4")
    print("   2. Considerar LSTM 68-72 units")
    print("   3. Mantener threshold 0.5")

print("="*80)
print("\nCOMPARACION CON ITERACIONES PREVIAS:")
print("V2: MCC 0.8885 (overfitting Delta ~0.16) <- Mejor score publico pero mas overfitting")
print("V3: MCC 0.8733 (overfitting Delta <0.08) <- Base de V6")
print(f"V6: MCC {mcc_val:.4f} (overfitting Delta {overfitting_delta:.4f}) <- MODELO ACTUAL")
print("="*80)

# ANÁLISIS EXHAUSTIVO Y CONCLUSIONES FINALES

## 1. Análisis de Métricas del Modelo Final

Este apartado presenta el análisis completo de las métricas obtenidas por el modelo V6,
incluyendo interpretación de resultados y comparación con iteraciones previas.

In [ ]:
# ==================================================================================
# ANALISIS DETALLADO DE METRICAS FINALES
# ==================================================================================
# Aqui conecto los resultados con todo el proceso experimental de las 6 iteraciones

print("\n" + "="*90)
print(" "*25 + "ANALISIS DE METRICAS FINALES - MODELO V6")
print("="*90)

# Recopilo todas las metricas importantes
final_metrics = {
    'MCC Validacion': mcc_val,
    'Accuracy Validacion': val_acc_final,
    'Train Loss': train_loss_final,
    'Validation Loss': val_loss_final,
    'Overfitting Delta': overfitting_delta,
    'Epochs Entrenados': final_epoch,
    'Separacion de Clases': separation
}

print("\nMETRICAS PRINCIPALES:")
print("-" * 90)
for metric, value in final_metrics.items():
    print(f"{metric:.<50} {value:.4f}")

print("\n" + "="*90)
print("INTERPRETACION:")
print("="*90)

# 1. Analizo MCC
print(f"\n1. MCC (Matthews Correlation Coefficient): {mcc_val:.4f}")
print("-" * 90)
print("   MCC es la metrica oficial de la competicion")
print("   Es mejor que accuracy para datasets desbalanceados")
print(f"\n   Mi MCC de {mcc_val:.4f} significa:")
if mcc_val >= 0.87:
    print("   -> EXCELENTE: Correlacion muy fuerte")
    print("   -> El modelo distingue claramente SPAM de NOT SPAM")
    print("   -> Estoy en el rango competitivo")
elif mcc_val >= 0.80:
    print("   -> BUENO: Correlacion fuerte, funciona bien")
    print("   -> Hay margen de mejora pero es solido")
else:
    print("   -> MEJORABLE: Deberia revisar la arquitectura")

# 2. Analizo Overfitting
print(f"\n2. OVERFITTING (Delta: {overfitting_delta:.4f})")
print("-" * 90)
print(f"   Train Loss: {train_loss_final:.4f}")
print(f"   Val Loss: {val_loss_final:.4f}")
print(f"   Diferencia (Delta): {overfitting_delta:.4f}")
print("\n   Que significa:")
if overfitting_delta < 0.05:
    print("   -> EXCELENTE: Casi sin overfitting")
    print("   -> El modelo generaliza perfectamente")
    print("   -> Incluso podria reducir un poco la regularizacion")
elif overfitting_delta < 0.10:
    print("   -> OPTIMO: Overfitting controlado")
    print("   -> Este es el balance perfecto")
    print("   -> No tocar la regularizacion, esta bien asi")
elif overfitting_delta < 0.15:
    print("   -> ACEPTABLE: Ligero overfitting")
    print("   -> Funciona bien pero hay margen de mejora")
    print("   -> Considerar aumentar regularizacion")
else:
    print("   -> PROBLEMATICO: Overfitting severo")
    print("   -> El modelo memoriza en lugar de aprender")
    print("   -> URGENTE: Aumentar regularizacion")

# 3. Analizo Separacion de Clases
print(f"\n3. SEPARACION DE CLASES: {separation:.4f}")
print("-" * 90)
print("   Es la diferencia entre prob media de SPAM vs NOT SPAM")
print("   Mide que tan bien distingue el modelo ambas clases")
print("\n   Mi separacion de {separation:.4f} significa:")
if separation > 0.6:
    print("   -> EXCELENTE: El modelo separa muy claramente")
    print("   -> Alta confianza en las predicciones")
elif separation > 0.4:
    print("   -> BUENO: Separacion clara")
    print("   -> El modelo tiene buena capacidad discriminativa")
elif separation > 0.3:
    print("   -> ACEPTABLE: Separacion moderada")
    print("   -> Funciona pero podria mejorar")
else:
    print("   -> INSUFICIENTE: Baja separacion")
    print("   -> El modelo no distingue bien las clases")

# Comparacion con otras iteraciones
print("\n" + "="*90)
print("COMPARACION CON MIS ITERACIONES PREVIAS:")
print("="*90)

# Data de todas mis iteraciones
comparison_data = {
    'V1 (Baseline)': {'MCC': 0.8665, 'Delta': 0.184, 'Status': 'Overfitting severo'},
    'V2 (Mejor Publico)': {'MCC': 0.8885, 'Delta': 0.166, 'Status': 'Overfitting moderado'},
    'V3 (Regularizacion)': {'MCC': 0.8733, 'Delta': 0.08, 'Status': 'Controlado'},
    'V4 (DistilBERT)': {'MCC': 0.6456, 'Delta': 'N/A', 'Status': 'FRACASO TOTAL'},
    'V5 (CNN+LSTM)': {'MCC': 0.83, 'Delta': 0.242, 'Status': 'Overfitting severo'},
    'V6 (ESTE MODELO)': {'MCC': mcc_val, 'Delta': overfitting_delta, 'Status': 'SELECCIONADO'}
}

print("\n{:<25} {:<12} {:<12} {:<25}".format("Iteracion", "MCC", "Delta", "Estado"))
print("-" * 90)
for iteration, metrics in comparison_data.items():
    delta_str = f"{metrics['Delta']:.3f}" if isinstance(metrics['Delta'], float) else metrics['Delta']
    marker = " <- FINAL" if "ESTE" in iteration else ""
    print(f"{iteration:<25} {metrics['MCC']:.4f}      {delta_str:<12} {metrics['Status']:<25}{marker}")

print("\n" + "="*90)
print("CONCLUSION:")
print("V6 no es el que tiene mejor MCC (ese es V2 con 0.8885)")
print("PERO V6 tiene mucho mejor overfitting (0.09 vs 0.166)")
print("Prefiero estabilidad a score publico alto")
print("="*90)

## 2. Curvas de Aprendizaje y Diagnóstico de Overfitting/Underfitting

Las curvas de aprendizaje son fundamentales para diagnosticar el comportamiento del modelo
durante el entrenamiento.

In [ ]:
# ==================================================================================
# CURVAS DE APRENDIZAJE
# ==================================================================================
# Visualizo como evoluciono el modelo durante el entrenamiento

# Config de estilo
plt.style.use('seaborn-v0_8-darkgrid')
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Curvas de Aprendizaje - Modelo V6 Final', fontsize=16, fontweight='bold')

# Rango de epochs para los graficos
epochs_range = range(1, len(history.history['loss']) + 1)

# --- GRAFICO 1: Loss ---
ax1 = axes[0, 0]
# Ploteo train loss y val loss
ax1.plot(epochs_range, history.history['loss'], 'b-o', label='Train Loss', linewidth=2, markersize=6)
ax1.plot(epochs_range, history.history['val_loss'], 'r-s', label='Validation Loss', linewidth=2, markersize=6)
# Lineas horizontales para valores finales
ax1.axhline(y=train_loss_final, color='b', linestyle='--', alpha=0.3, label=f'Final Train: {train_loss_final:.4f}')
ax1.axhline(y=val_loss_final, color='r', linestyle='--', alpha=0.3, label=f'Final Val: {val_loss_final:.4f}')
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('Evolucion del Loss', fontsize=14, fontweight='bold')
ax1.legend(loc='best', fontsize=10)
ax1.grid(True, alpha=0.3)

# Añado etiqueta de diagnostico segun overfitting
if overfitting_delta < 0.10:
    ax1.text(0.5, 0.95, 'Overfitting: CONTROLADO', 
             transform=ax1.transAxes, ha='center', va='top',
             bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7),
             fontsize=11, fontweight='bold')
elif overfitting_delta < 0.15:
    ax1.text(0.5, 0.95, 'Overfitting: LEVE', 
             transform=ax1.transAxes, ha='center', va='top',
             bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7),
             fontsize=11, fontweight='bold')
else:
    ax1.text(0.5, 0.95, 'Overfitting: SEVERO', 
             transform=ax1.transAxes, ha='center', va='top',
             bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.7),
             fontsize=11, fontweight='bold')

# --- GRAFICO 2: Accuracy ---
ax2 = axes[0, 1]
# Ploteo train accuracy y val accuracy
ax2.plot(epochs_range, history.history['accuracy'], 'b-o', label='Train Accuracy', linewidth=2, markersize=6)
ax2.plot(epochs_range, history.history['val_accuracy'], 'r-s', label='Validation Accuracy', linewidth=2, markersize=6)
# Lineas finales
ax2.axhline(y=train_acc_final, color='b', linestyle='--', alpha=0.3)
ax2.axhline(y=val_acc_final, color='r', linestyle='--', alpha=0.3)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Accuracy', fontsize=12)
ax2.set_title('Evolucion de Accuracy', fontsize=14, fontweight='bold')
ax2.legend(loc='best', fontsize=10)
ax2.grid(True, alpha=0.3)
ax2.set_ylim([0.85, 1.0])  # Zoom para ver mejor

# --- GRAFICO 3: Learning Rate ---
ax3 = axes[1, 0]
# Si hay historial de LR, lo ploteo
if 'lr' in history.history:
    ax3.plot(epochs_range, history.history['lr'], 'g-o', linewidth=2, markersize=6)
    ax3.set_xlabel('Epoch', fontsize=12)
    ax3.set_ylabel('Learning Rate', fontsize=12)
    ax3.set_title('Evolucion del Learning Rate', fontsize=14, fontweight='bold')
    ax3.set_yscale('log')  # Escala logaritmica
    ax3.grid(True, alpha=0.3)
else:
    # Si no hay historial, muestro mensaje
    ax3.text(0.5, 0.5, 'Learning Rate history no disponible\n(ReduceLROnPlateau aplicado)', 
             ha='center', va='center', fontsize=12, transform=ax3.transAxes)
    ax3.set_title('Evolucion del Learning Rate', fontsize=14, fontweight='bold')

# --- GRAFICO 4: Overfitting Delta ---
ax4 = axes[1, 1]
# Calculo delta en cada epoch
delta_history = [val - train for val, train in zip(history.history['val_loss'], history.history['loss'])]
# Ploteo
ax4.plot(epochs_range, delta_history, 'purple', linewidth=2, marker='o', markersize=6)
# Lineas de referencia
ax4.axhline(y=0, color='black', linestyle='-', linewidth=1, alpha=0.3)
ax4.axhline(y=0.10, color='orange', linestyle='--', linewidth=1, alpha=0.5, label='Threshold 0.10')
ax4.axhline(y=0.15, color='red', linestyle='--', linewidth=1, alpha=0.5, label='Threshold 0.15')
# Area bajo la curva
ax4.fill_between(epochs_range, 0, delta_history, alpha=0.3, color='purple')
ax4.set_xlabel('Epoch', fontsize=12)
ax4.set_ylabel('Overfitting Delta (Val Loss - Train Loss)', fontsize=12)
ax4.set_title('Evolucion del Overfitting', fontsize=14, fontweight='bold')
ax4.legend(loc='best', fontsize=10)
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# DIAGNOSTICO BASADO EN LAS CURVAS
print("\n" + "="*90)
print("DIAGNOSTICO BASADO EN CURVAS:")
print("="*90)

# Analizo velocidad de convergencia
num_epochs = len(history.history['loss'])
if num_epochs < 5:
    print("\nCONVERGENCIA PREMATURA:")
    print("   Se detuvo muy rapido. Considerar:")
    print("   - Aumentar patience de EarlyStopping")
    print("   - Reducir learning rate inicial")
elif num_epochs > 30:
    print("\nCONVERGENCIA LENTA:")
    print("   Necesito muchas epochs. Considerar:")
    print("   - Aumentar learning rate inicial")
    print("   - Ajustar ReduceLROnPlateau")
else:
    print(f"\nCONVERGENCIA ADECUADA: {num_epochs} epochs")
    print("   El modelo converge en tiempo razonable")

# Analizo tendencias de las ultimas 3 epochs
train_loss_trend = history.history['loss'][-3:] if num_epochs >= 3 else history.history['loss']
val_loss_trend = history.history['val_loss'][-3:] if num_epochs >= 3 else history.history['val_loss']

# Train loss debe ir bajando
if all(train_loss_trend[i] > train_loss_trend[i+1] for i in range(len(train_loss_trend)-1)):
    print("\nTRAIN LOSS: Descendente (BIEN)")
else:
    print("\nTRAIN LOSS: Con oscilaciones")

# Val loss idealmente baja o se estanca
if all(val_loss_trend[i] > val_loss_trend[i+1] for i in range(len(val_loss_trend)-1)):
    print("VAL LOSS: Descendente (BIEN)")
elif len(set(val_loss_trend)) == 1:
    print("VAL LOSS: Estancada (convergencia prematura?)")
else:
    print("VAL LOSS: Con oscilaciones (normal con ReduceLROnPlateau)")

print("\n" + "="*90)